In [16]:
import os
import random
import shutil
import xml.etree.ElementTree as ET

# 각 디렉터리 설정
base_dir12 = '/Users/psjj/Downloads/aihub_dataset/all_images'
xml_files = [os.path.join(base_dir12, filename) for filename in os.listdir(base_dir12) if filename.endswith('.xml')]
output_base = '/Users/psjj/Downloads/aihub_dataset/yolo_split'

# 분할 폴더 생성
train_dir = os.path.join(output_base, 'train')
val_dir = os.path.join(output_base, 'val')
test_dir = os.path.join(output_base, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# 유니크 클래스 저장
unique_classes = set()

# Step 1: XML 파일을 읽고 이미지 주석을 YOLO 형식으로 변환
annotations = []
for xml_file in xml_files:
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    for image in root.findall('image'):
        filename = image.get('name')
        width = int(image.get('width'))
        height = int(image.get('height'))

        yolo_annotations = []
        for box in image.findall('box'):
            label = box.get('label')
            # 유니크 클래스에 추가
            unique_classes.add(label)
            
            xmin = float(box.get('xtl'))
            ymin = float(box.get('ytl'))
            xmax = float(box.get('xbr'))
            ymax = float(box.get('ybr'))
            
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            yolo_annotations.append(f"{label} {x_center} {y_center} {box_width} {box_height}")

        annotations.append((filename, yolo_annotations))

# Step 2: 이미지 파일을 8:1:1 비율로 분할
random.shuffle(annotations)
total_images = len(annotations)
train_split = int(0.8 * total_images)
val_split = int(0.9 * total_images)

train_annotations = annotations[:train_split]
val_annotations = annotations[train_split:val_split]
test_annotations = annotations[val_split:]

# Step 3: YOLO 형식으로 각 분할 폴더에 저장
def save_annotations(annotations, output_dir):
    for filename, yolo_annotations in annotations:
        # 이미지 파일을 올바른 위치로 복사 (이미지 경로를 기반으로 수정 필요)
        image_path = os.path.join(base_dir12, filename)
        if os.path.exists(image_path):
            shutil.copy(image_path, os.path.join(output_dir, filename))
        
        # YOLO 형식으로 저장
        if yolo_annotations:
            yolo_file_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.txt')
            with open(yolo_file_path, 'w') as f:
                f.write("\n".join(yolo_annotations))

# 각각 train, val, test 폴더에 저장
save_annotations(train_annotations, train_dir)
save_annotations(val_annotations, val_dir)
save_annotations(test_annotations, test_dir)

# unique_classes에 있는 모든 클래스를 확인하여 최종 리스트로 출력
print("Unique classes found:", unique_classes)

Unique classes found: {'bench', 'car', 'potted_plant', 'fire_hydrant', 'movable_signage', 'motorcycle', 'stroller', 'table', 'bus', 'bicycle', 'wheelchair', 'traffic_sign', 'barricade', 'carrier', 'parking_meter', 'kiosk', 'tree_trunk', 'person', 'pole', 'stop', 'chair', 'traffic_light', 'truck', 'bollard'}


In [17]:
import os
import shutil

# 경로 설정
all_images_dir = '/Users/psjj/Downloads/aihub_dataset/all_images'  # 모든 이미지가 모여 있는 폴더
split_dir = '/Users/psjj/Downloads/aihub_dataset/yolo_split'  # train, val, test 폴더가 위치한 경로

# train, val, test 폴더 각각에서 .txt 파일과 동일한 이름의 이미지를 찾아 복사
for split in ['train', 'val', 'test']:
    split_path = os.path.join(split_dir, split)
    
    for txt_file in os.listdir(split_path):
        if txt_file.endswith('.txt'):
            # .txt 파일명에서 .jpg 파일명으로 변경
            image_file = txt_file.replace('.txt', '.jpg')
            src_image_path = os.path.join(all_images_dir, image_file)
            dst_image_path = os.path.join(split_path, image_file)
            
            # 이미지가 all_images에 있는 경우 해당 split 폴더로 복사
            if os.path.exists(src_image_path):
                shutil.copy(src_image_path, dst_image_path)
            else:
                print(f"Warning: {image_file} not found in {all_images_dir}")

In [18]:
import os

# 클래스 이름과 ID 매핑
class_mapping = {
    'bench': 0, 'car': 1, 'potted_plant': 2, 'fire_hydrant': 3, 
    'movable_signage': 4, 'motorcycle': 5, 'stroller': 6, 'table': 7, 
    'bus': 8, 'bicycle': 9, 'wheelchair': 10, 'traffic_sign': 11, 
    'barricade': 12, 'carrier': 13, 'parking_meter': 14, 'kiosk': 15, 
    'tree_trunk': 16, 'person': 17, 'pole': 18, 'stop': 19, 
    'chair': 20, 'traffic_light': 21, 'truck': 22, 'bollard': 23
}

# 데이터셋 경로
dataset_path = "/Users/psjj/Downloads/aihub_dataset/yolo_split/"

# 각 분할 폴더에서 라벨 파일 변환
for split in ['train', 'val', 'test']:
    split_path = os.path.join(dataset_path, split)
    for txt_file in os.listdir(split_path):
        if txt_file.endswith('.txt'):
            txt_path = os.path.join(split_path, txt_file)
            new_lines = []
            with open(txt_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    if parts[0] in class_mapping:
                        # 클래스 이름을 ID로 변환
                        class_id = class_mapping[parts[0]]
                        new_line = f"{class_id} " + " ".join(parts[1:])
                        new_lines.append(new_line)
                    else:
                        print(f"Warning: Unrecognized class '{parts[0]}' in {txt_file}")
            
            # 변환된 내용을 다시 파일에 저장
            with open(txt_path, 'w') as file:
                file.write("\n".join(new_lines))